# Sex-Specific Alzheimer's Disease Modelling Experimentations

## Set-up and Cleaning

In [189]:
# === Load and clean data ===
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns




In [191]:
# Load OASIS dataset
df = pd.read_csv("oasis_longitudinal.csv")
print(df)

    Subject ID         MRI ID        Group  Visit  MR Delay M/F Hand  Age  \
0    OAS2_0001  OAS2_0001_MR1  Nondemented      1         0   M    R   87   
1    OAS2_0001  OAS2_0001_MR2  Nondemented      2       457   M    R   88   
2    OAS2_0002  OAS2_0002_MR1     Demented      1         0   M    R   75   
3    OAS2_0002  OAS2_0002_MR2     Demented      2       560   M    R   76   
4    OAS2_0002  OAS2_0002_MR3     Demented      3      1895   M    R   80   
..         ...            ...          ...    ...       ...  ..  ...  ...   
368  OAS2_0185  OAS2_0185_MR2     Demented      2       842   M    R   82   
369  OAS2_0185  OAS2_0185_MR3     Demented      3      2297   M    R   86   
370  OAS2_0186  OAS2_0186_MR1  Nondemented      1         0   F    R   61   
371  OAS2_0186  OAS2_0186_MR2  Nondemented      2       763   F    R   63   
372  OAS2_0186  OAS2_0186_MR3  Nondemented      3      1608   F    R   65   

     EDUC  SES  MMSE  CDR  eTIV   nWBV    ASF  
0      14  2.0  27.0  0.0  

In [156]:
# Fill missing SES and MMSE values
df['SES'] = df['SES'].fillna(df['SES'].median())
df['MMSE'] = df['MMSE'].fillna(df['MMSE'].mean())

In [157]:
# Convert labels to binary classification: demented = 1, nondemented = 0
df['Group'] = df['Group'].replace("Converted", "Demented")
df['Group'] = df['Group'].map({"Demented": 1, "Nondemented": 0})

In [158]:
# Define features and dropping missing values
my_features = ['Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
df_clean = df.dropna(subset=my_features + ['Group', 'M/F'])

## Repeated Train/Test Evaluation Function

In [160]:

# === REPEATED TRAIN/TEST EVALUATION ===

def repeated_evaluation(model, X, y, label, sex=None, sex_filter=None, groups=None, n_splits=5, n_repeats=5):
    metrics = []
    for repeat in range(n_repeats):
        sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42 + repeat)
        for train_idx, test_idx in sgkf.split(X, y, groups):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            scaler = StandardScaler() 
            X_train_scaled = scaler.fit_transform(X_train) 
            X_test_scaled = scaler.transform(X_test)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            y_proba = model.predict_proba(X_test_scaled) [:, 1]
        if sex is not None and sex_filter is not None:
            test_sex = sex.iloc[test_idx]
            mask = test_sex == sex_filter
            if not mask.any():
                continue
            y_test = y_test[mask]
            y_pred = y_pred[mask]
            y_proba = y_proba[mask]

        metrics.append([
            accuracy_score(y_test, y_pred),
            f1_score(y_test, y_pred),
            precision_score(y_test, y_pred),
            recall_score(y_test, y_pred),
            roc_auc_score(y_test, y_proba)

        ])
        
    metrics = np.array(metrics)
    means = metrics.mean(axis=0)
    stds = metrics.std(axis=0)
    return[label,
           means[0], stds[0],
           means[1], stds[1],
           means[2], stds[2],
           means[3], stds[3],
           means[4], stds[4]
          ]


In [161]:
results = []
X_all = df_clean[my_features]
y_all = df_clean['Group']
sex_all = df_clean['M/F']
groups_all = df_clean['Subject ID']

## Logistic Regression and Random Forest Classifier Models

In [163]:

# === General models ===
results.append(repeated_evaluation(LogisticRegression(max_iter=1000), X_all, y_all, "Gen-LR-All", groups=groups_all))
results.append(repeated_evaluation(RandomForestClassifier(n_estimators=100, random_state=42), X_all, y_all, "Gen-RF-All", groups=groups_all))
results.append(repeated_evaluation(LogisticRegression(max_iter=1000), X_all, y_all, "Gen-LR-M", sex=sex_all, sex_filter='M', groups=groups_all))
results.append(repeated_evaluation(LogisticRegression(max_iter=1000), X_all, y_all, "Gen-LR-F", sex=sex_all, sex_filter='F', groups=groups_all))
results.append(repeated_evaluation(RandomForestClassifier(n_estimators=100, random_state=42), X_all, y_all, "Gen-RF-M", sex=sex_all, sex_filter='M', groups=groups_all))
results.append(repeated_evaluation(RandomForestClassifier(n_estimators=100, random_state=42), X_all, y_all, "Gen-RF-F", sex=sex_all, sex_filter='F', groups=groups_all))


In [164]:

# === Sex specific models ===
groups_f = df_female['Subject ID']
groups_m = df_male['Subject ID']

df_male = df_clean[df_clean['M/F'] == 'M']
X_m, y_m = df_male[my_features], df_male['Group']
results.append(repeated_evaluation(LogisticRegression(max_iter=1000), X_m, y_m, "Male-LR-M", groups=groups_m))
results.append(repeated_evaluation(RandomForestClassifier(n_estimators=100, random_state=42), X_m, y_m, "Male-RF-M", groups=groups_m))

df_female = df_clean[df_clean['M/F'] == 'F']
X_f, y_f = df_female[my_features], df_female['Group']
results.append(repeated_evaluation(LogisticRegression(max_iter=1000), X_f, y_f, "Female-LR-F", groups=groups_f))
results.append(repeated_evaluation(RandomForestClassifier(n_estimators=100, random_state=42), X_f, y_f, "Female-RF-F", groups=groups_f))

In [165]:

# === Results Table ===
columns =['Model', 'Accuracy_mean', 'Accuracy_std', 'F1_mean', 'F1_std', 'Precision_mean', 'Precision_std', 'Recall_mean', 'Recall_std', 'AUC_mean', 'AUC_std']
results_table = pd.DataFrame(results, columns=columns)
print(results_table)


         Model  Accuracy_mean  Accuracy_std   F1_mean    F1_std  \
0   Gen-LR-All       0.940433      0.013530  0.935406  0.016686   
1   Gen-RF-All       0.945530      0.015396  0.941044  0.020566   
2     Gen-LR-M       0.965453      0.011637  0.973939  0.009822   
3     Gen-LR-F       0.918739      0.024582  0.858285  0.050277   
4     Gen-RF-M       0.960190      0.012633  0.970433  0.009508   
5     Gen-RF-F       0.932655      0.029246  0.879259  0.067712   
6    Male-LR-M       0.970289      0.034285  0.976566  0.026787   
7    Male-RF-M       0.930390      0.042489  0.944774  0.034610   
8  Female-LR-F       0.959723      0.035108  0.921754  0.072858   
9  Female-RF-F       0.936847      0.027467  0.891209  0.061842   

   Precision_mean  Precision_std  Recall_mean  Recall_std  AUC_mean   AUC_std  
0        0.986406       0.016686     0.889599    0.020809  0.971773  0.013940  
1        0.981644       0.015480     0.903995    0.029399  0.959873  0.020312  
2        0.982213     

## XGBOOST models

In [167]:
xgb_results = []
xgb_results.append(repeated_evaluation(XGBClassifier(eval_metric='logloss'), X_all, y_all, 'Gen-XGB-All', groups=groups_all))
xgb_results.append(repeated_evaluation(XGBClassifier(eval_metric='logloss'), X_all, y_all, 'Gen-XGB-M', sex=sex_all, sex_filter='M', groups=groups_all))
xgb_results.append(repeated_evaluation(XGBClassifier(eval_metric='logloss'), X_all, y_all, 'Gen-XGB-F', sex=sex_all, sex_filter='F', groups=groups_all))
xgb_results.append(repeated_evaluation(XGBClassifier(eval_metric='logloss'), X_m, y_m, 'Male-XGB-M', groups=groups_m))
xgb_results.append(repeated_evaluation(XGBClassifier(eval_metric='logloss'), X_f, y_f, 'Female-XGB-F', groups=groups_f))


In [168]:
xgb_results_table = pd.DataFrame(xgb_results, columns=columns)
print(xgb_results_table)

          Model  Accuracy_mean  Accuracy_std   F1_mean    F1_std  \
0   Gen-XGB-All       0.940637      0.029733  0.934190  0.039977   
1     Gen-XGB-M       0.947964      0.033635  0.961388  0.025751   
2     Gen-XGB-F       0.932364      0.048222  0.875929  0.103211   
3    Male-XGB-M       0.958673      0.031574  0.966515  0.025021   
4  Female-XGB-F       0.913392      0.029098  0.867011  0.025363   

   Precision_mean  Precision_std  Recall_mean  Recall_std  AUC_mean   AUC_std  
0        0.956288       0.048440     0.913946    0.041486  0.949987  0.020266  
1        0.957609       0.045652     0.966375    0.017141  0.974155  0.016054  
2        0.949206       0.063014     0.817803    0.136534  0.907991  0.056663  
3        0.971292       0.037185     0.965035    0.052842  0.976713  0.036987  
4        0.913621       0.078898     0.843312    0.108519  0.902926  0.067514  


## Support Vector Machine and Voting Classifier Models 

In [197]:
svm_vote_results = []
svm_model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)

# === SVM general models ===
svm_vote_results.append(repeated_evaluation(svm_model, X_all, y_all, 'Gen-SVM-All', groups=groups_all))
svm_vote_results.append(repeated_evaluation(svm_model, X_all, y_all, 'Gen-SVM-M', groups=groups_all))
svm_vote_results.append(repeated_evaluation(svm_model, X_all, y_all, 'Gen-SVM-F', groups=groups_all))

# === SVM sex-specific models ===
svm_vote_results.append(repeated_evaluation(svm_model, X_m, y_m, 'Male-SVM-M', groups=groups_m))
svm_vote_results.append(repeated_evaluation(svm_model, X_f, y_f, 'Female-SVM-F', groups=groups_f))

In [198]:
# === Voting Classifier Set-up ===

voting_estimators =[
    ('lr', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)),
]
voting_clf = VotingClassifier(estimators=voting_estimators, voting='soft')

    

In [203]:
# === VC general models ===
svm_vote_results.append(repeated_evaluation(voting_clf, X_all, y_all, 'Gen-VOTE-All', groups=groups_all))
svm_vote_results.append(repeated_evaluation(voting_clf, X_all, y_all, 'Gen-VOTE-M', sex=sex_all, sex_filter='M', groups=groups_all))
svm_vote_results.append(repeated_evaluation(voting_clf, X_all, y_all, 'Gen-VOTE-F', sex=sex_all, sex_filter='F', groups=groups_all))
                                         
# === VC sex-specific models
svm_vote_results.append(repeated_evaluation(voting_clf, X_m, y_m, 'Male-VOTE-M', groups=groups_m))
svm_vote_results.append(repeated_evaluation(voting_clf, X_f, y_f, 'Female-VOTE-F', groups=groups_f))                                          


In [205]:
# === SVM and VC results ===

svm_vote_results_table = pd.DataFrame(svm_vote_results, columns=columns)
print(svm_vote_results_table)

            Model  Accuracy_mean  Accuracy_std   F1_mean    F1_std  \
0     Gen-SVM-All       0.937801      0.015605  0.932609  0.019460   
1       Gen-SVM-M       0.937801      0.015605  0.932609  0.019460   
2       Gen-SVM-F       0.937801      0.015605  0.932609  0.019460   
3      Male-SVM-M       0.969595      0.048442  0.976078  0.038790   
4    Female-SVM-F       0.954845      0.030574  0.911228  0.064064   
5    Gen-VOTE-All       0.940433      0.013530  0.935406  0.016686   
6      Gen-VOTE-M       0.965453      0.011637  0.973939  0.009822   
7      Gen-VOTE-F       0.918739      0.024582  0.858285  0.050277   
8    Gen-VOTE-All       0.940433      0.013530  0.935406  0.016686   
9      Gen-VOTE-M       0.965453      0.011637  0.973939  0.009822   
10     Gen-VOTE-F       0.918739      0.024582  0.858285  0.050277   
11    Male-VOTE-M       0.975845      0.036338  0.981444  0.028344   
12  Female-VOTE-F       0.954845      0.030574  0.912231  0.064658   

    Precision_mean 

In [170]:
#### TODO
#from scipy.stats import ttest_rel
#import pandas as pd

# Extract AUC scores for each model from the earlier results
# Define model pairs to compare
# Perform t-tests
# Bonferroni correction

# Evaluate all models with cross validation??